In [3]:
# CSL Paper: Dimensional speech emotion recognition from acoustic and text
# Changelog:
# 2019-09-01: initial version
# 2019-10-06: optimizer MTL parameters with linear search (in progress)
# 2019-12-21: update file for CSL journal
#             this python use following feature: mean, std, silence

import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

#from keras_self_attention import SeqSelfAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from attention_helper import AttentionDecoder
#from read_csv import load_features

import random as rn
import tensorflow as tf

import gensim
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

In [4]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [5]:
# x_text = np.load('../improv_loso/data/x_text.npy')

In [6]:
# x_text.shape

In [7]:
# load data - label
list_path_train = '../improv_loso/data/data_mspin_train.csv'
list_train = pd.read_csv(list_path_train, index_col=None)

# vad_train = [list_train['v'], list_train['a'], list_train['d']]
# vad_train = np.array(vad_train).T

list_path_test = '../improv_loso/data/data_mspin_test.csv'
list_test = pd.read_csv(list_path_test, index_col=None)

## Text processing train
list_file = pd.concat([list_train, list_test])
text = list_file['transcript']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
nb_words = len(word_index) + 1
print('Found %s unique tokens' % len(word_index))

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_text = []
x_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

# x_text = np.load('improv_loso/data/x_text.npy')
# x_train_text = x_text[:5744] 
# x_test_text = x_text[5744:]

# vad = np.load('improv_loso/data/vad.npy')
# vad_train = vad[:5744]
# vad_test = vad[5744:]

EMBEDDING_DIM = 300
# nb_words = 3438
# MAX_SEQUENCE_LENGTH = 300

#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
file_loc =  '../../../dimensionalSM/data/GoogleNews-vectors-negative300.bin'
print (file_loc)

word_vectors = KeyedVectors.load_word2vec_format(file_loc, binary=True)

g_word_embedding_matrix = {}
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=nb_words:
        continue
    try:
        gembedding_vector = word_vectors[word]
        g_word_embedding_matrix[i] = gembedding_vector
    except KeyError:
        g_word_embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), EMBEDDING_DIM)

del(word_vectors)

# print input size
print('Feature shape: ', x_text.shape)
# print('Label shape: ', vad_train.shape)


Found 3427 unique tokens
../../../dimensionalSM/data/GoogleNews-vectors-negative300.bin
Feature shape:  (7166, 300)


In [8]:
vad = np.load('../improv_loso/data/vad.npy')

In [9]:
# standardization
scaled_feature = False

# set Dropout
do = 0.3

# standardized label
scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad


# Concordance correlation coefficient (CCC)-based loss function
# using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [10]:
# split train/test
split = 5744
earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)

In [11]:
# API model, if use RNN, first two rnn layer must return_sequences=True
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model1 = text_model1()

In [12]:
# main function for for LOSO
def main(time):
    model_1 = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [x_text[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [x_text[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict(x_text[4596:], batch_size=32)
    np.save('5744npy-W2V/text_W2V-5744-npy'+str(time)+'.npy',  
             np.array(predict).reshape(3, 2570).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('5744npy-W2V/text_W2V-5744.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 90s 20ms/step - loss: 2.5181 - v_loss: 0.7287 - a_loss: 0.8827 - d_loss: 0.9044 - v_ccc: 0.2706 - a_ccc: 0.1163 - d_ccc: 0.0949 - val_loss: 2.6763 - val_v_loss: 0.8673 - val_a_loss: 0.8769 - val_d_loss: 0.9319 - val_v_ccc: 0.1326 - val_a_ccc: 0.1233 - val_d_ccc: 0.0678
Epoch 2/50
4595/4595 [==============================] - 88s 19ms/step - loss: 2.0184 - v_loss: 0.5693 - a_loss: 0.7060 - d_loss: 0.7425 - v_ccc: 0.4304 - a_ccc: 0.2943 - d_ccc: 0.2570 - val_loss: 2.6952 - val_v_loss: 0.8282 - val_a_loss: 0.9155 - val_d_loss: 0.9514 - val_v_ccc: 0.1714 - val_a_ccc: 0.0847 - val_d_ccc: 0.0487
Epoch 3/50
4595/4595 [==============================] - 88s 19ms/step - loss: 1.7612 - v_loss: 0.4919 - a_loss: 0.6181 - d_loss: 0.6521 - v_ccc: 0.5085 - a_ccc: 0.3819 - d_ccc: 0.3484 - val_loss: 2.7020 - val_v_loss: 0

4595/4595 [==============================] - 90s 20ms/step - loss: 2.5086 - v_loss: 0.7350 - a_loss: 0.8712 - d_loss: 0.9010 - v_ccc: 0.2646 - a_ccc: 0.1282 - d_ccc: 0.0985 - val_loss: 2.6733 - val_v_loss: 0.8037 - val_a_loss: 0.9659 - val_d_loss: 0.9046 - val_v_ccc: 0.1965 - val_a_ccc: 0.0346 - val_d_ccc: 0.0956
Epoch 2/50
4595/4595 [==============================] - 89s 19ms/step - loss: 2.0091 - v_loss: 0.5734 - a_loss: 0.6951 - d_loss: 0.7400 - v_ccc: 0.4267 - a_ccc: 0.3044 - d_ccc: 0.2598 - val_loss: 2.6541 - val_v_loss: 0.8424 - val_a_loss: 0.9001 - val_d_loss: 0.9120 - val_v_ccc: 0.1575 - val_a_ccc: 0.1005 - val_d_ccc: 0.0878
Epoch 3/50
4595/4595 [==============================] - 89s 19ms/step - loss: 1.7633 - v_loss: 0.5024 - a_loss: 0.6142 - d_loss: 0.6459 - v_ccc: 0.4970 - a_ccc: 0.3854 - d_ccc: 0.3542 - val_loss: 2.6759 - val_v_loss: 0.8505 - val_a_loss: 0.9124 - val_d_loss: 0.9126 - val_v_ccc: 0.1490 - val_a_ccc: 0.0880 - val_d_ccc: 0.0872
Epoch 4/50
4595/4595 [===========

4595/4595 [==============================] - 91s 20ms/step - loss: 2.4966 - v_loss: 0.7285 - a_loss: 0.8666 - d_loss: 0.9001 - v_ccc: 0.2706 - a_ccc: 0.1327 - d_ccc: 0.1001 - val_loss: 2.5837 - val_v_loss: 0.7968 - val_a_loss: 0.8913 - val_d_loss: 0.8952 - val_v_ccc: 0.2030 - val_a_ccc: 0.1090 - val_d_ccc: 0.1043
Epoch 2/50
4595/4595 [==============================] - 89s 19ms/step - loss: 1.9837 - v_loss: 0.5612 - a_loss: 0.6877 - d_loss: 0.7343 - v_ccc: 0.4391 - a_ccc: 0.3118 - d_ccc: 0.2655 - val_loss: 2.6716 - val_v_loss: 0.8287 - val_a_loss: 0.9116 - val_d_loss: 0.9313 - val_v_ccc: 0.1707 - val_a_ccc: 0.0889 - val_d_ccc: 0.0688
Epoch 3/50
4595/4595 [==============================] - 90s 19ms/step - loss: 1.7526 - v_loss: 0.4961 - a_loss: 0.6055 - d_loss: 0.6506 - v_ccc: 0.5039 - a_ccc: 0.3943 - d_ccc: 0.3492 - val_loss: 2.6186 - val_v_loss: 0.8162 - val_a_loss: 0.8934 - val_d_loss: 0.9078 - val_v_ccc: 0.1830 - val_a_ccc: 0.1067 - val_d_ccc: 0.0917
Epoch 4/50
4595/4595 [===========

4595/4595 [==============================] - 88s 19ms/step - loss: 1.7890 - v_loss: 0.4945 - a_loss: 0.6300 - d_loss: 0.6658 - v_ccc: 0.5058 - a_ccc: 0.3705 - d_ccc: 0.3347 - val_loss: 2.6401 - val_v_loss: 0.8103 - val_a_loss: 0.9082 - val_d_loss: 0.9212 - val_v_ccc: 0.1891 - val_a_ccc: 0.0920 - val_d_ccc: 0.0787
Epoch 4/50
4595/4595 [==============================] - 89s 19ms/step - loss: 1.5943 - v_loss: 0.4465 - a_loss: 0.5547 - d_loss: 0.5937 - v_ccc: 0.5530 - a_ccc: 0.4461 - d_ccc: 0.4067 - val_loss: 2.7394 - val_v_loss: 0.8073 - val_a_loss: 0.9440 - val_d_loss: 0.9875 - val_v_ccc: 0.1921 - val_a_ccc: 0.0560 - val_d_ccc: 0.0125
Epoch 5/50
4595/4595 [==============================] - 89s 19ms/step - loss: 1.4533 - v_loss: 0.3948 - a_loss: 0.5135 - d_loss: 0.5431 - v_ccc: 0.6049 - a_ccc: 0.4857 - d_ccc: 0.4560 - val_loss: 2.7352 - val_v_loss: 0.8309 - val_a_loss: 0.9345 - val_d_loss: 0.9689 - val_v_ccc: 0.1686 - val_a_ccc: 0.0653 - val_d_ccc: 0.0309
Epoch 6/50
4595/4595 [===========

4595/4595 [==============================] - 87s 19ms/step - loss: 2.0176 - v_loss: 0.5735 - a_loss: 0.6985 - d_loss: 0.7443 - v_ccc: 0.4262 - a_ccc: 0.3009 - d_ccc: 0.2553 - val_loss: 2.7263 - val_v_loss: 0.8303 - val_a_loss: 0.9307 - val_d_loss: 0.9653 - val_v_ccc: 0.1695 - val_a_ccc: 0.0695 - val_d_ccc: 0.0347
Epoch 3/50
4595/4595 [==============================] - 87s 19ms/step - loss: 1.7613 - v_loss: 0.4979 - a_loss: 0.6122 - d_loss: 0.6519 - v_ccc: 0.5025 - a_ccc: 0.3882 - d_ccc: 0.3480 - val_loss: 2.7114 - val_v_loss: 0.8694 - val_a_loss: 0.8969 - val_d_loss: 0.9449 - val_v_ccc: 0.1304 - val_a_ccc: 0.1034 - val_d_ccc: 0.0548
Epoch 4/50
4595/4595 [==============================] - 87s 19ms/step - loss: 1.5804 - v_loss: 0.4453 - a_loss: 0.5547 - d_loss: 0.5816 - v_ccc: 0.5548 - a_ccc: 0.4457 - d_ccc: 0.4191 - val_loss: 2.6884 - val_v_loss: 0.8316 - val_a_loss: 0.9157 - val_d_loss: 0.9410 - val_v_ccc: 0.1681 - val_a_ccc: 0.0844 - val_d_ccc: 0.0591
Epoch 5/50
4595/4595 [===========

4595/4595 [==============================] - 90s 20ms/step - loss: 2.5144 - v_loss: 0.7287 - a_loss: 0.8768 - d_loss: 0.9076 - v_ccc: 0.2708 - a_ccc: 0.1231 - d_ccc: 0.0918 - val_loss: 2.6871 - val_v_loss: 0.8142 - val_a_loss: 0.9316 - val_d_loss: 0.9422 - val_v_ccc: 0.1856 - val_a_ccc: 0.0688 - val_d_ccc: 0.0584
Epoch 2/50
4595/4595 [==============================] - 83s 18ms/step - loss: 2.0289 - v_loss: 0.5685 - a_loss: 0.7050 - d_loss: 0.7541 - v_ccc: 0.4310 - a_ccc: 0.2947 - d_ccc: 0.2455 - val_loss: 2.6518 - val_v_loss: 0.8388 - val_a_loss: 0.9066 - val_d_loss: 0.9061 - val_v_ccc: 0.1608 - val_a_ccc: 0.0936 - val_d_ccc: 0.0937
Epoch 3/50
4595/4595 [==============================] - 84s 18ms/step - loss: 1.7747 - v_loss: 0.5043 - a_loss: 0.6146 - d_loss: 0.6570 - v_ccc: 0.4962 - a_ccc: 0.3858 - d_ccc: 0.3433 - val_loss: 2.6632 - val_v_loss: 0.8586 - val_a_loss: 0.8931 - val_d_loss: 0.9109 - val_v_ccc: 0.1409 - val_a_ccc: 0.1071 - val_d_ccc: 0.0887
Epoch 4/50
4595/4595 [===========

4595/4595 [==============================] - 88s 19ms/step - loss: 2.0094 - v_loss: 0.5671 - a_loss: 0.7017 - d_loss: 0.7405 - v_ccc: 0.4325 - a_ccc: 0.2982 - d_ccc: 0.2600 - val_loss: 2.6968 - val_v_loss: 0.8002 - val_a_loss: 0.9424 - val_d_loss: 0.9543 - val_v_ccc: 0.1996 - val_a_ccc: 0.0577 - val_d_ccc: 0.0458
Epoch 3/50
4595/4595 [==============================] - 88s 19ms/step - loss: 1.7664 - v_loss: 0.4927 - a_loss: 0.6190 - d_loss: 0.6542 - v_ccc: 0.5070 - a_ccc: 0.3806 - d_ccc: 0.3459 - val_loss: 2.6909 - val_v_loss: 0.8502 - val_a_loss: 0.9082 - val_d_loss: 0.9319 - val_v_ccc: 0.1490 - val_a_ccc: 0.0920 - val_d_ccc: 0.0681
Epoch 4/50
4595/4595 [==============================] - 87s 19ms/step - loss: 1.5754 - v_loss: 0.4363 - a_loss: 0.5518 - d_loss: 0.5870 - v_ccc: 0.5641 - a_ccc: 0.4478 - d_ccc: 0.4127 - val_loss: 2.7335 - val_v_loss: 0.8475 - val_a_loss: 0.9117 - val_d_loss: 0.9734 - val_v_ccc: 0.1519 - val_a_ccc: 0.0882 - val_d_ccc: 0.0265
Epoch 5/50
4595/4595 [===========

4595/4595 [==============================] - 86s 19ms/step - loss: 1.6015 - v_loss: 0.4382 - a_loss: 0.5667 - d_loss: 0.5967 - v_ccc: 0.5623 - a_ccc: 0.4330 - d_ccc: 0.4033 - val_loss: 2.7763 - val_v_loss: 0.8537 - val_a_loss: 0.9506 - val_d_loss: 0.9712 - val_v_ccc: 0.1455 - val_a_ccc: 0.0495 - val_d_ccc: 0.0286
Epoch 5/50
4595/4595 [==============================] - 87s 19ms/step - loss: 1.4478 - v_loss: 0.3976 - a_loss: 0.5133 - d_loss: 0.5378 - v_ccc: 0.6033 - a_ccc: 0.4868 - d_ccc: 0.4622 - val_loss: 2.8056 - val_v_loss: 0.8657 - val_a_loss: 0.9508 - val_d_loss: 0.9887 - val_v_ccc: 0.1339 - val_a_ccc: 0.0492 - val_d_ccc: 0.0112
Epoch 6/50
4595/4595 [==============================] - 86s 19ms/step - loss: 1.3273 - v_loss: 0.3646 - a_loss: 0.4730 - d_loss: 0.4904 - v_ccc: 0.6361 - a_ccc: 0.5267 - d_ccc: 0.5099 - val_loss: 2.7516 - val_v_loss: 0.8339 - val_a_loss: 0.9379 - val_d_loss: 0.9788 - val_v_ccc: 0.1654 - val_a_ccc: 0.0620 - val_d_ccc: 0.0209
Epoch 7/50
4595/4595 [===========

4595/4595 [==============================] - 87s 19ms/step - loss: 1.3150 - v_loss: 0.3576 - a_loss: 0.4673 - d_loss: 0.4905 - v_ccc: 0.6426 - a_ccc: 0.5328 - d_ccc: 0.5096 - val_loss: 2.7704 - val_v_loss: 0.8477 - val_a_loss: 0.9447 - val_d_loss: 0.9776 - val_v_ccc: 0.1517 - val_a_ccc: 0.0556 - val_d_ccc: 0.0224
Epoch 7/50
4595/4595 [==============================] - 87s 19ms/step - loss: 1.2216 - v_loss: 0.3343 - a_loss: 0.4385 - d_loss: 0.4474 - v_ccc: 0.6656 - a_ccc: 0.5608 - d_ccc: 0.5520 - val_loss: 2.7664 - val_v_loss: 0.8468 - val_a_loss: 0.9366 - val_d_loss: 0.9828 - val_v_ccc: 0.1525 - val_a_ccc: 0.0636 - val_d_ccc: 0.0175
Epoch 8/50
4595/4595 [==============================] - 87s 19ms/step - loss: 1.1367 - v_loss: 0.3147 - a_loss: 0.4095 - d_loss: 0.4125 - v_ccc: 0.6851 - a_ccc: 0.5905 - d_ccc: 0.5877 - val_loss: 2.8396 - val_v_loss: 0.8574 - val_a_loss: 0.9643 - val_d_loss: 1.0179 - val_v_ccc: 0.1420 - val_a_ccc: 0.0359 - val_d_ccc: -0.0175
Epoch 9/50
4595/4595 [==========

4595/4595 [==============================] - 88s 19ms/step - loss: 1.2049 - v_loss: 0.3288 - a_loss: 0.4323 - d_loss: 0.4432 - v_ccc: 0.6710 - a_ccc: 0.5673 - d_ccc: 0.5568 - val_loss: 2.7424 - val_v_loss: 0.8751 - val_a_loss: 0.9124 - val_d_loss: 0.9543 - val_v_ccc: 0.1242 - val_a_ccc: 0.0877 - val_d_ccc: 0.0457
Epoch 8/50
4595/4595 [==============================] - 89s 19ms/step - loss: 1.1193 - v_loss: 0.3082 - a_loss: 0.3987 - d_loss: 0.4115 - v_ccc: 0.6913 - a_ccc: 0.6009 - d_ccc: 0.5885 - val_loss: 2.8461 - val_v_loss: 0.8680 - val_a_loss: 0.9531 - val_d_loss: 1.0247 - val_v_ccc: 0.1314 - val_a_ccc: 0.0470 - val_d_ccc: -0.0245
Epoch 9/50
4595/4595 [==============================] - 89s 19ms/step - loss: 1.0356 - v_loss: 0.2926 - a_loss: 0.3604 - d_loss: 0.3829 - v_ccc: 0.7079 - a_ccc: 0.6396 - d_ccc: 0.6169 - val_loss: 2.7579 - val_v_loss: 0.8505 - val_a_loss: 0.9369 - val_d_loss: 0.9691 - val_v_ccc: 0.1487 - val_a_ccc: 0.0629 - val_d_ccc: 0.0306
Epoch 10/50
4595/4595 [=========

In [13]:
# hist1 = model1.fit(x_text[:split], vad[:split].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate(x_text[split:], vad[split:].T.tolist())
# print(eval_metrik1[-3:])

# # # make prediction
# # predict = model1.predict(x_text[4596:], batch_size=32)
# # np.save('result_mspin_loso/result_ter_glove', 
# #          np.array(predict).reshape(3, 2570).T)